# Speedup Pandas

- This INCREDIBLE trick will speed up your data processes.
https://www.youtube.com/watch?v=u4rsA5ZiTls
- pandas-data-storage-compare.py github
https://gist.github.com/RobMulla/738491f7bf7cfe79168c7e55c622efa5

In [87]:
import pandas as pd
import numpy as np

def get_dataset(size):
    # Create Fake Dataset
    df = pd.DataFrame()
    df['size'] = np.random.choice(['big','medium','small'], size)
    df['age'] = np.random.randint(1, 50, size)
    df['team'] = np.random.choice(['red','blue','yellow','green'], size)
    df['win'] = np.random.choice(['yes','no'], size)
    dates = pd.date_range('2020-01-01', '2022-12-31')
    df['date'] = np.random.choice(dates, size)
    df['prob'] = np.random.uniform(0, 1, size)
    return df

def set_dtypes(df):
    df['size'] = df['size'].astype('category')
    df['team'] = df['team'].astype('category')
    df['age'] = df['age'].astype('int16')
    df['win'] = df['win'].map({'yes':True, 'no': False})
    df['prob'] = df['prob'].astype('float32')
    return df

In [88]:
df = get_dataset(10_000)

In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   size    10000 non-null  object        
 1   age     10000 non-null  int64         
 2   team    10000 non-null  object        
 3   win     10000 non-null  object        
 4   date    10000 non-null  datetime64[ns]
 5   prob    10000 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 468.9+ KB


In [90]:
df.head()

,size,age,team,win,date,prob
0,medium,33,yellow,yes,2022-05-05,0.129045
1,small,36,red,no,2020-08-02,0.119849
2,big,3,green,yes,2022-02-16,0.453556
3,small,20,red,no,2021-03-28,0.682364
4,medium,10,blue,no,2021-08-28,0.994119


# Measure execution time

- Measure execution time with timeit in Python
https://note.nkmk.me/en/python-timeit-measure/
- Jupyter Magic - Timing(%%time %time %timeit)
https://blog.csdn.net/shuibuzhaodeshiren/article/details/86650688

In [91]:
%time df = get_dataset(3)
%time df.head()

CPU times: user 0 ns, sys: 2.67 ms, total: 2.67 ms
Wall time: 2.72 ms
CPU times: user 0 ns, sys: 88 µs, total: 88 µs
Wall time: 90.6 µs


,size,age,team,win,date,prob
0,big,22,blue,yes,2022-08-31,0.953732
1,small,26,green,yes,2021-08-02,0.934422
2,medium,20,red,no,2020-11-21,0.396267


In [92]:
%%time
df = get_dataset(3)
df.head()

CPU times: user 4.56 ms, sys: 61 µs, total: 4.62 ms
Wall time: 4.61 ms


,size,age,team,win,date,prob
0,big,12,blue,yes,2022-06-07,0.656868
1,small,28,blue,yes,2020-08-21,0.299710
2,small,47,green,yes,2020-04-03,0.326247


In [93]:
# %timeit df = get_dataset(3)
# %timeit df.head()
# 1.49 ms ± 5.63 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
# 40.7 µs ± 6.39 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
# timeit would determine the below automatically
# 單趟測試次數(-r): e.g. 7 runs
# 測試多少趟(-n): e.g. 1,000 loops

In [94]:
%timeit -r 3 -n 1 df = get_dataset(3)
%timeit -r 3 -n 1 df.head()

5.73 ms ± 2.23 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
99.7 µs ± 35.3 µs per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [95]:
%%timeit -r 3 -n 1
df = get_dataset(3)
df.head()

8.13 ms ± 3.59 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


# CSV

In [96]:
df = get_dataset(1_000_000)
df = set_dtypes(df)
%time df.to_csv('test.csv', index=False)
# df.to_csv('test.csv')  # index=True by default
%time df_csv = pd.read_csv('test.csv')
# df_csv = pd.read_csv('test.csv', index_col=[0])
df_csv.head()

CPU times: user 2.14 s, sys: 63.2 ms, total: 2.21 s
Wall time: 2.24 s
CPU times: user 352 ms, sys: 12 ms, total: 364 ms
Wall time: 364 ms


,size,age,team,win,date,prob
0,medium,45,green,False,2020-12-11,0.260603
1,big,34,yellow,True,2020-09-10,0.829594
2,medium,8,yellow,True,2022-05-24,0.380245
3,medium,29,blue,True,2021-12-09,0.620741
4,medium,14,green,False,2022-07-20,0.501858


In [97]:
!ls -GFlash test.csv

40M -rw-rw-r-- 1 wmnlab 40M  二  11 18:43 test.csv


In [98]:
df.dtypes

size          category
age              int16
team          category
win               bool
date    datetime64[ns]
prob           float32
dtype: object

In [99]:
# dtypes are not totally the same as input
df_csv.dtypes

size     object
age       int64
team     object
win        bool
date     object
prob    float64
dtype: object

# Pickle

Need not to set dtypes again (while reading data)!

In [100]:
df = get_dataset(1_000_000)
df = set_dtypes(df)
%time df.to_pickle('test.pickle')
%time df_pickle = pd.read_pickle('test.pickle')
df_pickle.head()

CPU times: user 0 ns, sys: 15.3 ms, total: 15.3 ms
Wall time: 21.2 ms
CPU times: user 0 ns, sys: 3.09 ms, total: 3.09 ms
Wall time: 3.03 ms


,size,age,team,win,date,prob
0,small,14,yellow,True,2022-09-13,0.677967
1,big,30,red,True,2020-04-27,0.284421
2,small,11,blue,True,2021-01-21,0.907483
3,medium,20,green,True,2022-06-28,0.402508
4,big,29,red,False,2021-03-05,0.097007


In [101]:
!ls -GFlash test.pickle

17M -rw-rw-r-- 1 wmnlab 17M  二  11 18:43 test.pickle


In [102]:
df.dtypes

size          category
age              int16
team          category
win               bool
date    datetime64[ns]
prob           float32
dtype: object

In [103]:
df_pickle.dtypes

size          category
age              int16
team          category
win               bool
date    datetime64[ns]
prob           float32
dtype: object

# Parquet

In [104]:
!pip install pyarrow
!pip install fastparquet

In [105]:
df = get_dataset(1_000_000)
df = set_dtypes(df)
%time df.to_parquet('test.parquet')
%time df_parquet = pd.read_parquet('test.parquet')
df_parquet.head()

CPU times: user 94.6 ms, sys: 23.4 ms, total: 118 ms
Wall time: 120 ms
CPU times: user 29.1 ms, sys: 45.5 ms, total: 74.5 ms
Wall time: 27.1 ms


,size,age,team,win,date,prob
0,medium,3,green,True,2022-02-01,0.669044
1,medium,45,blue,False,2020-08-12,0.066560
2,small,13,blue,True,2022-02-24,0.992863
3,small,1,yellow,False,2021-12-07,0.476879
4,medium,39,blue,True,2021-11-08,0.321545


In [106]:
!ls -GFlash test.parquet

7.1M -rw-rw-r-- 1 wmnlab 7.1M  二  11 18:43 test.parquet


In [107]:
df.dtypes

size          category
age              int16
team          category
win               bool
date    datetime64[ns]
prob           float32
dtype: object

In [108]:
df_parquet.dtypes

size          category
age              int16
team          category
win               bool
date    datetime64[ns]
prob           float32
dtype: object

## Read in specific columns

In [109]:
df_parquet = pd.read_parquet('test.parquet', columns=['date', 'win'])
df_parquet.head()

,date,win
0,2022-02-01,True
1,2020-08-12,False
2,2022-02-24,True
3,2021-12-07,False
4,2021-11-08,True


# Feather

In [110]:
df = get_dataset(1_000_000)
df = set_dtypes(df)
%time df.to_feather('test.feather')
%time df_feather = pd.read_feather('test.feather')
df_feather.head()

CPU times: user 60.8 ms, sys: 5.45 ms, total: 66.3 ms
Wall time: 38.8 ms
CPU times: user 21 ms, sys: 7.21 ms, total: 28.2 ms
Wall time: 13.4 ms


,size,age,team,win,date,prob
0,big,6,yellow,True,2022-07-30,0.313523
1,small,44,red,False,2020-12-17,0.358667
2,big,36,red,True,2021-09-20,0.743402
3,big,5,blue,True,2021-11-04,0.206697
4,small,13,red,False,2022-12-03,0.521272


In [111]:
!ls -GFlash test.feather

9.8M -rw-rw-r-- 1 wmnlab 9.8M  二  11 18:43 test.feather


In [112]:
df.dtypes

size          category
age              int16
team          category
win               bool
date    datetime64[ns]
prob           float32
dtype: object

In [113]:
df_feather.dtypes

size          category
age              int16
team          category
win               bool
date    datetime64[ns]
prob           float32
dtype: object

In [114]:
!ls -GFlash test.csv test.pickle test.parquet test.feather

 40M -rw-rw-r-- 1 wmnlab  40M  二  11 18:43 test.csv
9.8M -rw-rw-r-- 1 wmnlab 9.8M  二  11 18:43 test.feather
7.1M -rw-rw-r-- 1 wmnlab 7.1M  二  11 18:43 test.parquet
 17M -rw-rw-r-- 1 wmnlab  17M  二  11 18:43 test.pickle


: 

# Speed & Space

- Speed: Feather > Pickle > Parquet > Csv
- Space: Parquet < Feather < Pickle < Csv
- Parquet can read specific columns only!